# Carry Trade

## Imports

In [167]:
# <include-carry_trade/utils.py>

In [168]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio

from carry_trade import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"

## Summary

Here we analyze three different carry trade strategies involving cross-currency fixed-float and basis swaps. In each we borrow Japanese Yen at three-month Libor + 50bps and use 80% leverage to purchase 5-year government bonds in one of the following markets:
* Thailand
* Romania
* Indonesia

Our investment period spans from 2015-01-01 to 2021-04-22. We mark to market back to USD on a weekly basis.

### Results

* None of the strategies was good.
* The one involving the Thai Bhat was the best, producing a cumulative return of 39.9% over the period with a Sharpe ratio of just 0.0236.
* Profit was negatively impacted in all of the strategies early on by unfavorable fx rate movements.
* All of the strategies experienced catastrophic losses as a result of extreme jumps in yields at the onset of the pandemic. The strategy involving the Indonesian Rupiah lost over 40% in a single day.

### Implementation Details

* Yield curves are from [Quandl Global Yield Curves (YC)](https://www.quandl.com/data/YC-Global-Yield-Curves)
* Foreign exchange rates are from [Quandl Foreign Exchange Rates (CUR)](https://www.quandl.com/data/CUR-Foreign-Exchange-Rates)
* 3-month Libor in Japanese Yen is from [FRED Economic Data (JPY3MTD156N)](https://fred.stlouisfed.org/series/JPY3MTD156N)
* Weekly returns are calculated as follows:
    * Exchange $2.0 million USD capital for Japanese Yen
    * Borrow $8.0 million USD value Japanse Yen at 3-month Libor +50bps (borrow market)
    * Convert $10.0 million USD value Japanese Yen to currency of market in which bond will be purchased (lend market)
    * Calculate zero coupon bond curve from swap rate yield curve in lend market
    * Calculate notional amount of $10.0 million USD value bonds purchased using zero coupon bond curve
    * Calculate zero coupon bond curve from swap rate yield curve in lend market one week later
    * Calculate current value of notional amount of purchased bonds from updated zero coupon bond curve with rates interpolated back one week to align with cash flow of purchased bond
    * Convert back to Japanese Yen
    * Repay borrowings
    * Convert capital and profit back to USD
    * Calculate return


### Parameters

In [169]:
start_date = "2014-12-01"
tickers = ["THA", "ROU", "JPN", "IDN"]
libors = ["JPY3MTD156N"]
currencies = ["THB", "RON", "JPY", "IDR"]

In [170]:
dfs_yc = utils.load_yc(tickers)
dfs_fx = utils.load_fx(currencies)
dfs_libor = utils.load_libor(libors)

strategies = [
    {"yc_L": "THA", "fx_B": "JPY", "fx_L": "THB", "libor": "JPY3MTD156N", "leverage": 0.8},
    {"yc_L": "ROU", "fx_B": "JPY", "fx_L": "RON", "libor": "JPY3MTD156N", "leverage": 0.8},
    {"yc_L": "IDN", "fx_B": "JPY", "fx_L": "IDR", "libor": "JPY3MTD156N", "leverage": 0.8},
]

date_range = pd.date_range("2015-01-01", "2021-04-26", freq="7D")

## Thai Baht

In [171]:
df_ret_0, df_profit_0 = utils.run_strategy(
    **strategies[0],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor   
)

**Distributions of returns**
* This doesn't appear to be a great strategy with a mean weekly return of 0.0012 (0.0624 annualized), standard deviation of 0.0513 and Sharpe ratio of just 0.0236.
* It starts out performing for the first quarter and then loses money for the next year and half, through the third quarter of 2016.
* It then enjoys a relatively long run of positive performance, up through the beginning of 2020 when the pandemic starts.
* It then has another precipitous drop before resuming relatively consistent positive performance, ending with a cumulative total return of 0.3987.
* The distribution of returns is consistent with that of a carry trade strategy, with some fatness in the tails (excess kurtosis of 1.53) and some negative skewness (-0.61).
* The fatness of the tails can also been see in the Q-Q plot, although the outliers and are not extreme (consistent with modest excess kurtosis)

**Drivers of profit and loss**
* As noted above, returns come primarily from two source: (i) changes in the value of the bond in the lend market resulting from changes in the yield curve there and (ii) foreign exchange movements.
* Profits result from decreases in rates in the lending market, driving up the price of the bond that we own.
* With respect to foreign exchange rates, profits results from decreases in the rate between the lend market and the borrow market, i.e., we get to convert back into more of the borrow market currency when we sell our bond, if the rate has declined. Less importantly, since it is on a smaller basis (just our capital as opposed to including the 80% leverage in the case of the lend market to borrow market fx rate), profits also result from decreases in the borrow market rate relative to dollars. However, that obviously has the opposite effect on the more important lend-borrow rate.
* To be able to discern the impact on those effects separately, the components chart below shows the lend-borrow fx rate and the borrow-home fx rates separately.

*2015Q1*

* For the initial period of positive perfomance, that looks to have been driven by rates coming down modestly, relatively flat borrow market fx and and favorble changes in the lend-borrow fx rate.

*2015Q2 - 2016Q3*

* Negative peformance driven by significant consistent unfavorable movements in the lend-borrow fx rate more than offsetting favorable movements in the lend market yields.
* Looks to be driven by unfavorable movents in the Japanese Yen, coinciding with the ocurrence of negative short term interest rates (albeit small)

*2016Q3 - 2020Q1*

* Relatively consistent positive perormance - exhibiting "normal" carry trade behavior - driven by, on average, declines in lend market yields and positive lend-borrw fx rate changes.
* This is actually quite a run, with nearly a 1.5x return on investment. If we were to look just at this period, this would look like a very attractive strategy.

*2020Q1*

* The conspicuous feature of this period is the dramatic spike in yields - up 52% during the week of 2020-03-19.
* As noted in class, the exchange rate moved in the wrong direction at the same time - there were several days with nearly 4% unfavorable changes during the same time frame.
* The result is several single day drop of near 20% with a maximum drop of 21.1% on 2020-04-02 (yields up 13.3% and lend-borrow fx rate up 3.8%).

In [172]:
utils.make_returns_chart(df_ret_0)

In [173]:
fig = utils.make_components_chart(
    **strategies[0],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor
)
fig.show()

For illustrative purposes, below are the returns for this strategy including only the favorable period with no big volaltility swings from 2016Q3 to 2020Q1.
* It looks well behaved and virtually gone are the outliers.
* I'm surprised the Sharpe ratio isn't higher, but the overall variability hasn't decreased that much - standard deviation of 0.0407 vs 0.0513 for the full period. Also on a relative basis is has increased alot - nearly nine times, but from a very small base.

In [174]:
df_ret_0a, df_profit_0a = utils.run_strategy(
    **strategies[0],
    date_range=pd.date_range("2016-10-01", "2020-02-28", freq="7D"),
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor   
)

In [175]:
utils.make_returns_chart(df_ret_0a)

## Romanian Leu

In [176]:
df_ret_1, df_profit_1 = utils.run_strategy(
    **strategies[1],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor   
)

**Distributions of returns**
* Overall this is poor strategy, with a negative cumulative return, higher variability and more extreme outliers than the previous strategy.
* This strategy loses all of its money after 18 months.

**Drivers of profit and loss**
* It looks like the main reason for the poor performance through the first 18 months is that the lend-borrow exchange rate moves are unfavorable and there are no corresponding favorable movements in yields to compensate for them like there were in the previous strategy.
* Leaving aside that we had already lost all our money by then, then losses that result from the pandemic volatility are even more extreme in this case with a maximu single day loss of over 40%.


In [177]:
utils.make_returns_chart(df_ret_1)

In [178]:
fig = utils.make_components_chart(
    **strategies[1],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor
)
fig.show()

## Indonesian Rupiah

In [179]:
df_ret_2, df_profit_2 = utils.run_strategy(
    **strategies[2],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor   
)

**Distributions of returns**
* This also appears to be a poor strategy, with a negative return and the highest variablity and most extreme outliers of the three strategies evaulated.

**Drivers of profit and loss**
* The dynamics of this strategy are similar to those of the Romanian Leu.

In [180]:
utils.make_returns_chart(df_ret_2)

In [181]:
fig = utils.make_components_chart(
    **strategies[2],
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor
)
fig.show()

## TL;DR

This is a bit of a dive into the weeds to make sure the mechanics of the returns calculations are being peformed correctly and also to understand the relative importance of the yield movements versus the fx movements. This looks at the week ending 2020-04-02, which was the single biggest down week in the Thai Bhat strategy, down 21.1%

In [182]:
row = df_profit_0.loc["2020-04-02"].to_dict()
row

{'fx_B0': 110.6239,
 'fx_B1': 107.485056,
 'fx_L0': 32.815,
 'fx_L1': 33.09,
 'zcb_L0': 0.00990235531649793,
 'zcb_L0t': 5.0,
 'zcb_L1': 0.011222842161769251,
 'zcb_L1t': 4.980821917808219,
 'V_L0': 328150000.00000006,
 'N_L0': 344806226.70035625,
 'V_L1': 326060719.2145738,
 'V_B0': 1106239000.0000002,
 'V_B1': 1059131298.4037092,
 'K_H0': 2000000.0,
 'K_H1': 1619426.5519234003,
 'r_H1': -0.211075074275663}

In [183]:
((row["V_B0"] * row["fx_L0"] / row["fx_B0"]) / np.exp(-row["zcb_L0"] * row["zcb_L0t"]) * np.exp(-row["zcb_L1"] * row["zcb_L1t"]) / row["fx_L1"] * row["fx_B1"] - row["V_B0"] * 0.8) / row["fx_B1"]

1620133.1132367735